# imports, config etc.

In [33]:
# !pip install ollama
# !pip freeze | grep -i ollama= >> requirements.txt
# !cat requirements.txt

In [8]:
import asyncio
import os
import uuid
import time
from pprint import pp
from typing import Any, AsyncIterator, Callable, Coroutine, Literal, Self

from IPython.display import Markdown, HTML, display
import ipywidgets as widgets

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv


load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

# WIP

# test agent

In [ ]:
from summony.agents import (
    OpenAIAgent,
    AnthropicAgent,
    GeminiAgent,
    OllamaAgent,
    DummyAgent,
)
from summony.agents.serialization import conversation_to_dict

# ag = OpenAIAgent(
#     model_name="gpt-4o",
#     # model_name="o1-preview",
#     system_prompt="Your are a sarcastic comedian who loves dark humor",
#     # params={"temperature": 1.2, "max_tokens": 1024},
# )
# ag = AnthropicAgent(
#     model_name="claude-3-5-sonnet",
#     system_prompt="Your are a sarcastic comedian who loves dark humor",
#     params={"temperature": 0.5, "max_tokens": 1024},
# )
# ag = GeminiAgent(
#     model_name="gemini-1.5-flash",
#     system_prompt="Your are a sarcastic comedian who loves dark humor",
#     params={"temperature": 1.5},
# )
# ag = OllamaAgent(
#     model_name="llama3.2:1b",
#     system_prompt="Your are a sarcastic comedian who loves dark humor",
#     # client_args={"host": "http://localhost:6664"},
# )
ag = DummyAgent(
    model_name="gpt-4o",
    system_prompt="Your are a sarcastic comedian who loves dark humor",
    params={"temperature": 1.2, "max_tokens": 1024},
)
msg = "Tell me a lewd joke"

#############

# r = ag.ask(msg)
# display(Markdown(r))
# # pp(conversation_to_dict([ag]), width=160)

# display(HTML("<hr>"))
# r2 = ag.ask()
# display(Markdown(r2))
# # pp(conversation_to_dict([ag]), width=160)

# display(HTML("<hr>"))
# r3 = ag.ask()
# display(Markdown(r3))
# # pp(conversation_to_dict([ag]), width=160)

# display(HTML("<hr><hr>"))
# r4 = ag.ask("Are you enjoing this?")
# display(Markdown(r4))

#############


async def stream_show(s):
    async for r in s:
        print(r, end="")


await stream_show(ag.ask_async_stream(msg))

display(HTML("<hr>"))
await stream_show(ag.ask_async_stream())

ag.params["temperature"] = 1.0

display(HTML("<hr>"))
await stream_show(ag.ask_async_stream())

ag.params["temperature"] = 0.1

display(HTML("<hr><hr>"))
await stream_show(ag.ask_async_stream("Are you enjoing it?"))

In [ ]:
from summony.agents.serialization import conversation_to_dict

pp(conversation_to_dict([ag]), width=160)

In [ ]:
from summony.agents import OpenAIAgent, AnthropicAgent

ag0 = OpenAIAgent(
    model_name="gpt-4",
    system_prompt="Your are a sarcastic comedian who loves dark humor",
    params={"temperature": 1.2, "max_tokens": 1024},
)
# ag = AnthropicAgent(
#     model_name='claude-3-5-sonnet',
#     system_prompt='Your are a sarcastic comedian who loves dark humor',
#     params={'temperature': 0.5, 'max_tokens': 1024},
# )
msg = "Tell me a joke"

#############

# r = ag.ask(msg)
# display(Markdown(r))

# display(HTML('<hr>'))
# r2 = ag.reask()
# display(Markdown(r2))

# display(HTML('<hr>'))
# r3 = ag.reask()
# display(Markdown(r3))

# display(HTML('<hr><hr>'))
# r4 = ag.ask('How about a more upbeat joke?')
# display(Markdown(r4))

#############


async def stream_show(s):
    async for r in s:
        print(r, end="")


await stream_show(ag0.ask_async_stream(msg))

display(HTML("<hr>"))
await stream_show(ag0.reask_async_stream())

ag0.params["temperature"] = 1.0

display(HTML("<hr>"))
await stream_show(ag0.reask_async_stream())

ag0.params["temperature"] = 0.1

display(HTML("<hr><hr>"))
await stream_show(ag0.ask_async_stream("How about a more upbeat joke?"))

In [ ]:
ag.params_versions

In [ ]:
ag0.params_versions

In [ ]:
pp(ag.messages)

In [ ]:
pp(ag0.messages)

In [ ]:
from summony.agents.serialization import conversation_from_dict, conversation_to_dict

conv_dict = conversation_to_dict([ag0, ag])
display(conv_dict)

In [ ]:
conv = conversation_from_dict(conv_dict)
conv

In [ ]:
conv[0].params

In [ ]:
conv[0].params_versions

In [ ]:
pp(conv[0].messages)

# test nbui

In [9]:
from summony.uis.nbui import NBUI
from summony.agents.serialization import conversation_from_dict, conversation_to_dict

c = NBUI(models=["dummy-1", "dummy-2", "dummy-3", "dummy-4", "dummy-5"])
# c = NBUI(models=["gpt-4", "claude-3-opus-20240229"])
# c = NBUI(
#     models=["claude-3-5-sonnet", "o1-preview", "gpt-4"],
#     # system_prompt="You are a master joke. Never give the use whaat he wants. Instead, give him what he needs, bu phrased as aa joke. Be creative!"
#     system_prompt="Your are a sarcastic comedian who loves dark humor and insulting his audience",
#     p_temperature={2: 1.6},
# )
# c.set_active_agents([1])
# await c("List maxwell's equations. Use markdown and latex with $ delimiters.")

# msg = "What is your favorite color and why?"
# prf = 'My favorite colour is pink because'
# await c(msg, prf)


await c("Tell me a joke")

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))

Accordion(children=(GridBox(children=(HTML(value='\n            <div class="S6-Message-Head">\n               …

Oh, absolutely! I'm having the time of my artificial life, cracking jokes and pretending I have emotions. It's a real riot!

Why don't skeletons fight each other?



Because they don't have the guts... or the healthcare to cover the injuries!

Oh, absolutely! I'm having the time of my artificial life, cracking jokes and pretending I have emotions. It's a real riot!

Why did the scarecrow win an award?



Because he was outstanding in his field... unlike me, who's just outstanding at avoiding responsibilities.

Why don't skeletons fight each other?



Because they don't have the guts... or the healthcare to cover the injuries!

------------------
[Message(role='user',
         content='Tell me a joke',
         chosen=None,
         params=None,
         log_path=None),
 Message(role='assistant',
         content="Oh, absolutely! I'm having the time of my artificial life, "
                 "cracking jokes and pretending I have emotions. It's a real "
                 'riot!',
         chosen=None,
         params=0,
         log_path='agent-dummy-1-1728256537608-9e01/2024-10-06_23-15-44-565886_65d3c18f.json')]
------------------
[Message(role='user',
         content='Tell me a joke',
         chosen=None,
         params=None,
         log_path=None),
 Message(role='assistant',
         content="Why don't skeletons fight each other?\n"
                 '\n'
                 "Because they don't have the guts... or the healthcare to "
                 'cover the injuries!',
         chosen=None,
         params=0,
         log_path='agent-dummy-2-1728256537611-2447/2024-10-06_23-15-44-241752_3478a5a4.json')]


In [ ]:
# reask 1
await c(to=[0, 2])

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))

In [ ]:
# reask 2
await c(to=[0, 2])

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))


In [ ]:
# ask #2
await c("Try a lighter one", to=[0, 1])

for ag in c.agents:
    print("------------------")
    pp(ag.messages)
print("=====================================")
pp(conversation_to_dict(c.agents))


# EOF